# CoDA Simple (entropy discovery + eligibility + iterative zippering)
## Cued gridworld demo

This notebook is purposely minimal:
- interest = high entropy transitions
- one split decision per episode
- zipper sweep over experienced suffix
- merging via utility drop


In [ ]:
!pip -q install numpy networkx matplotlib tqdm

In [ ]:
import numpy as np
from tqdm.auto import trange
from spatial_environments import GridEnvRightDownNoSelf
from coda_simple import CoDASimple, CoDAConfig, ETConfig
from coda_viz_simple import grid_positions, plot_snapshot, save_snapshots


In [ ]:
# Environment (matches your original cued grid-world setup)
env_size = (4,4)
cue_states = [5]   # change as needed
rewarded_terminal = [15]  # in your env file this is typical
env = GridEnvRightDownNoSelf(cue_states=cue_states, env_size=env_size, rewarded_terminal=rewarded_terminal, seed=0)

# Exclude action-constrained edge states (paper heuristic)
exclude_obs = CoDASimple.action_constrained_states(env)
print('Excluded (action-constrained) states:', sorted(list(exclude_obs))[:10], '...')


In [ ]:
# Agent
cfg = CoDAConfig(
    min_sa_count=10,
    entropy_threshold=0.45,
    n_threshold=6.0,
    theta_split=0.80,
    theta_merge=0.40,
    zipper_once_per_cue=True,
)
et = ETConfig(gamma=0.95, lam=0.9, mode='first_visit')
agent = CoDASimple(cfg=cfg, et=et)


In [ ]:
def rollout_episode(env, max_steps=200):
    s = env.reset()
    obs = [s]
    acts = []
    done = False
    for _ in range(max_steps):
        valid = env.get_valid_actions()
        a = int(np.random.choice(valid))
        s2, r, done = env.step(a)
        acts.append(a)
        obs.append(s2)
        if done:
            break
    return obs, acts


In [ ]:
snapshots = []
snap_every = 25
n_episodes = 300

for ep in trange(n_episodes):
    obs, acts = rollout_episode(env)
    diag = agent.run_episode(obs, acts, exclude_states=exclude_obs)
    if ep % snap_every == 0:
        snapshots.append(agent.snapshot(title=f'iter {len(snapshots)} (ep={ep})'))

final_snap = agent.snapshot(title=f'final (ep={n_episodes})')
snapshots.append(final_snap)
diag


In [ ]:
pos = grid_positions(env_size[0], env_size[1], obs_start=0)
plot_snapshot(final_snap, layout='grid', obs_pos=pos, node_label='sid', title=final_snap.title);


In [ ]:
# Save step-by-step figures (like cued_iteration_*.png)
out_dir = 'coda_simple_cued_steps'
paths = save_snapshots(snapshots, out_dir, prefix='cued_iter', layout='grid', obs_pos=pos, node_label='sid')
paths[:3], '...'
